`Creational design patterns provide various object creation mechanisms, which increase flexibility and reuse of existing code.`

# Singleton
- A ideia é garantir que a classe tenha apenas uma instancia e que seja acessivel globalmente
- Ele ja é aplicado caso voce use o \_\_init.py\_\_ para importar a classe

## 01 - Implementação Thread Safe

In [6]:
from threading import Lock, Thread

class SingletonMeta(type):
    """
    Implementação thread-safe do padrão Singleton.
    """

    _instancias = {}

    _bloqueio: Lock = Lock()
    """
    Agora temos um objeto de bloqueio que será usado para sincronizar as threads
    durante o primeiro acesso ao Singleton.
    """
    # __call__ permite que você faça um objeto Python se comportar como uma função.
    def __call__(cls, *args, **kwargs):
        """
        Mudanças possíveis no valor do argumento `__init__` não afetam a
        instância retornada.
        """
        # Agora, imagine que o programa acabou de ser iniciado. Como não há
        # instância de Singleton ainda, várias threads podem passar
        # simultaneamente pela condicional anterior e chegar a este ponto
        # quase ao mesmo tempo. A primeira delas adquirirá o bloqueio e
        # prosseguirá, enquanto o resto esperará aqui.
        with cls._bloqueio:
            # A primeira thread a adquirir o bloqueio atinge esta condicional,
            # entra e cria a instância Singleton. Depois de sair do bloco de
            # bloqueio, uma thread que poderia estar esperando a liberação do
            # bloqueio pode então entrar nesta seção. Mas como o campo Singleton
            # já está inicializado, a thread não criará um novo objeto.
            if cls not in cls._instancias:
                instancia = super().__call__(*args, **kwargs)
                cls._instancias[cls] = instancia
        return cls._instancias[cls]

class Singleton(metaclass=SingletonMeta):
    valor: str = None
    """
    Usaremos esta propriedade para provar que nosso Singleton realmente
    funciona.
    """

    def __init__(self, valor: str) -> None:
        self.valor = valor

    def alguma_logica_de_negocio(self):
        """
        Finalmente, todo singleton deve definir alguma lógica de negócio, que
        pode ser executada em sua instância.
        """

def teste_singleton(valor: str) -> None:
    singleton = Singleton(valor)
    print(singleton.valor)

if __name__ == "__main__":
    # Código do cliente.

    print("Se você ver o mesmo valor, então o singleton foi reutilizado (yay!)\n"
          "Se você ver valores diferentes, "
          "então 2 singletons foram criados (booo!!)\n\n"
          "RESULTADO:\n")

    processo1 = Thread(target=teste_singleton, args=("FOO",))
    processo2 = Thread(target=teste_singleton, args=("BAR",))
    processo1.start()
    processo2.start()
    ss = Singleton("Ola")
    print(ss.valor)


Se você ver o mesmo valor, então o singleton foi reutilizado (yay!)
Se você ver valores diferentes, então 2 singletons foram criados (booo!!)

RESULTADO:

FOO
FOO
FOO


## 02 - Implementação mais fácil

In [8]:
class SingletonMeta(type):
    """
    A classe Singleton pode ser implementada de diferentes maneiras em Python.
    Alguns métodos possíveis incluem: classe base, decorador, metaclass. Vamos
    usar a metaclass porque ela é mais adequada para esse propósito.
    """

    _instancias = {}

    def __call__(cls, *args, **kwargs):
        """
        Mudanças possíveis no valor do argumento `__init__` não afetam a
        instância retornada.
        """
        if cls not in cls._instancias:
            instancia = super().__call__(*args, **kwargs)
            cls._instancias[cls] = instancia
        return cls._instancias[cls]


class Singleton(metaclass=SingletonMeta):
    def alguma_logica_de_negocio(self):
        """
        Finalmente, todo singleton deve definir alguma lógica de negócio, que
        pode ser executada em sua instância.
        """

        # ...


if __name__ == "__main__":
    # Código do cliente.

    s1 = Singleton()
    s2 = Singleton()

    print(s1)
    print(s2)
    if id(s1) == id(s2):
        print("Singleton funciona, ambas as variáveis contêm a mesma instância.")
    else:
        print("Singleton falhou, as variáveis contêm instâncias diferentes.")


Singleton funciona, ambas as variáveis contêm a mesma instância.


## 03 - Singleton simplao criado por mim -Sem Meta Class

In [10]:
class SingletonClass:
    _instance = None
    _initialized = False

    def __init__(self):
        if not self._initialized:
            self.value = None
            self._initialized = True
            print('Ola')

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(SingletonClass, cls).__new__(cls)
        return cls._instance

    def set_value(self, value):
        self.value = value

    def get_value(self):
        return self.value

class ClasseFilha1(SingletonClass):
    def __init__(self):
        super().__init__()  # Chama o método __init__ da classe pai
        self.set_value(10)  # Define um valor específico para ClasseFilha1

class ClasseFilha2(SingletonClass):
    def __init__(self):
        super().__init__()  # Chama o método __init__ da classe pai
        self.set_value(120)  # Define um valor específico para ClasseFilha2


# Criando instâncias da classe SingletonClass
singleton1 = SingletonClass()
singleton1.set_value(10)

singleton2 = SingletonClass()
singleton2.set_value(20)


# Ambas as instâncias referem-se ao mesmo objeto
print(singleton1.get_value())  # Saída: 20
print(singleton2.get_value())

Ola
20
20
